In [1]:
import cv2
import os
from inference import get_model # Assuming 'inference.py' contains the get_model function
import math # Import math for isnan check
import numpy as np # Import numpy for array operations

2025-08-31 16:38:40.451711: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/vanthkrab/PycharmProjects/COE64-335/.venv/lib/python3.12/site-packages/inference/models/utils.py:370: ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM_ENABLED to False.
  warnings.warn(
/home/vanthkrab/PycharmProjects/COE64-335/.venv/lib/python3.12/site-packages/inference/models/utils.py:382: ModelDependencyMissing: Your `inference` configuration does not support SAM2 model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM2_ENABLED to False.
  warnings.warn(
/home/vanthkra

In [4]:
# 🚀 Load model
# Ensure your Roboflow model ID and API key are correct and active.
model = get_model(
    model_id="mangosteen-markersc-detection-c7wdu/1",
    api_key="jwFwJKYOs7A8qwQzEb9h"
)

KeyboardInterrupt: 

In [3]:
folder_path = "./data/raw/full"
for filename in sorted(os.listdir(folder_path)):
    print(filename)


i001_side.jpg
i001_under.jpg
i001_upper.jpg
i002_side.jpg
i002_under.jpg
i002_upper.jpg
i003_side.jpg
i003_under.jpg
i003_upper.jpg
i004_side.jpg
i004_under.jpg
i004_upper.jpg
i005_side.jpg
i005_under.jpg
i005_upper.jpg
i006_side.jpg
i006_under.jpg
i006_upper.jpg
i007_side.jpg
i007_under.jpg
i007_upper.jpg
i008_side.jpg
i008_under.jpg
i008_upper.jpg
i009_side.jpg
i009_under.jpg
i009_upper.jpg
i010_side.jpg
i010_under.jpg
i010_upper.jpg
i011_side.jpg
i011_under.jpg
i011_upper.jpg
i012_side.jpg
i012_under.jpg
i012_upper.jpg
i013_side.jpg
i013_under.jpg
i013_upper.jpg
i014_side.jpg
i014_under.jpg
i014_upper.jpg
i015_side.jpg
i015_under.jpg
i015_upper.jpg
i016_side.jpg
i016_under.jpg
i016_upper.jpg
i017_side.jpg
i017_under.jpg
i017_upper.jpg
i018_side.jpg
i018_under.jpg
i018_upper.jpg
i019_side.jpg
i019_under.jpg
i019_upper.jpg
i020_side.jpg
i020_under.jpg
i020_upper.jpg
i021_side.jpg
i021_under.jpg
i021_upper.jpg
i022_side.jpg
i022_under.jpg
i022_upper.jpg
i023_side.jpg
i023_under.jpg
i02

In [4]:
# Define physical dimensions of the markers in millimeters
MARKER_PHYSICAL_SIZE_MM = 30.0 # Both square and circle markers are 30x30mm

# Create directories to save the output images
output_normalized_folder = os.path.join(folder_path, "output_normalized")
output_clean_crops_folder = os.path.join(folder_path, "output_clean_crops")

if not os.path.exists(output_normalized_folder):
    os.makedirs(output_normalized_folder)
if not os.path.exists(output_clean_crops_folder):
    os.makedirs(output_clean_crops_folder)

# Define the final output image size for the regression model
OUTPUT_SIZE_PX = 224 # A common input size for many CNN backbones

# Define the physical size of the canvas in millimeters
# This is a fixed reference that allows for correct proportional scaling.
CANVAS_SIZE_MM = 100.0 # e.g., A 100mm x 100mm physical area

# Define a tolerance for filtering out misidentified markers
SIZE_TOLERANCE_MM = 5.0 # Mangosteen detections with dimensions within this tolerance of 30mm will be ignored.

GLOBAL_SCALE_FACTOR = OUTPUT_SIZE_PX / CANVAS_SIZE_MM


In [5]:
for filename in sorted(os.listdir(folder_path)):
    print(filename)

i001_side.jpg
i001_under.jpg
i001_upper.jpg
i002_side.jpg
i002_under.jpg
i002_upper.jpg
i003_side.jpg
i003_under.jpg
i003_upper.jpg
i004_side.jpg
i004_under.jpg
i004_upper.jpg
i005_side.jpg
i005_under.jpg
i005_upper.jpg
i006_side.jpg
i006_under.jpg
i006_upper.jpg
i007_side.jpg
i007_under.jpg
i007_upper.jpg
i008_side.jpg
i008_under.jpg
i008_upper.jpg
i009_side.jpg
i009_under.jpg
i009_upper.jpg
i010_side.jpg
i010_under.jpg
i010_upper.jpg
i011_side.jpg
i011_under.jpg
i011_upper.jpg
i012_side.jpg
i012_under.jpg
i012_upper.jpg
i013_side.jpg
i013_under.jpg
i013_upper.jpg
i014_side.jpg
i014_under.jpg
i014_upper.jpg
i015_side.jpg
i015_under.jpg
i015_upper.jpg
i016_side.jpg
i016_under.jpg
i016_upper.jpg
i017_side.jpg
i017_under.jpg
i017_upper.jpg
i018_side.jpg
i018_under.jpg
i018_upper.jpg
i019_side.jpg
i019_under.jpg
i019_upper.jpg
i020_side.jpg
i020_under.jpg
i020_upper.jpg
i021_side.jpg
i021_under.jpg
i021_upper.jpg
i022_side.jpg
i022_under.jpg
i022_upper.jpg
i023_side.jpg
i023_under.jpg
i02

In [ ]:
for filename in sorted(os.listdir(folder_path)):
    print(filename)
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        print(f"⚠️ Skipping non-image file: {filename}")
        continue

    filepath = os.path.join(folder_path, filename)

    # Load the original image once
    original_image = cv2.imread(filepath)
    if original_image is None:
        print(f"⚠️ Failed to load image: {filename}")
        continue

    # Create a copy for drawing annotations
    display_image = original_image.copy()

    h_img, w_img = display_image.shape[:2]

    # --- Dynamic Scaling Parameters ---
    scale_factor = min(h_img, w_img) / 720.0
    scaled_font_scale = max(0.4, 0.75 * scale_factor)
    scaled_font_thickness = max(1, int(1 * scale_factor))
    scaled_line_thickness_boxes = max(1, int(2 * scale_factor))
    scaled_line_thickness_arrows = max(1, int(2 * scale_factor))
    scaled_tip_length = max(0.01, 0.03 * scale_factor)

    # 🔍 Inference
    result = model.infer(display_image)[0]

    mangosteen_boxes = []
    potential_markers = []

    print(f"\n📷 File: {filename}")
    if not result.predictions:
        print("    → No detections.")
        continue

    class_names_detected = set()

    for pred in result.predictions:
        x, y, w, h = pred.x, pred.y, pred.width, pred.height
        label = pred.class_name.strip().lower()
        confidence = pred.confidence

        class_names_detected.add(label)

        x1, y1 = int(x - w / 2), int(y - h / 2)
        x2, y2 = int(x + w / 2), int(y + h / 2)

        x1, x2 = max(0, min(x1, w_img - 1)), max(0, min(x2, w_img - 1))
        y1, y2 = max(0, min(y1, h_img - 1)), max(0, min(y2, h_img - 1))

        w_clamped = x2 - x1
        h_clamped = y2 - y1

        cv2.rectangle(display_image, (x1, y1), (x2, y2), (0, 0, 255), scaled_line_thickness_boxes // 2 + 1)
        cv2.putText(display_image, label, (x1, max(0, y1 - int(10 * scale_factor))),
                                 cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale * 0.8, (0, 0, 255), scaled_font_thickness)

        if label in ["marker_square", "marker_circle"]:
            if w_clamped > 0:
                ppm_w = w_clamped / MARKER_PHYSICAL_SIZE_MM
                ppm_error_w = abs((w_clamped + 1) / MARKER_PHYSICAL_SIZE_MM - ppm_w)
            else:
                ppm_w = 0
                ppm_error_w = float('inf')

            if h_clamped > 0:
                ppm_h = h_clamped / MARKER_PHYSICAL_SIZE_MM
                ppm_error_h = abs((h_clamped + 1) / MARKER_PHYSICAL_SIZE_MM - ppm_h)
            else:
                ppm_h = 0
                ppm_error_h = float('inf')

            if ppm_error_w <= ppm_error_h:
                best_ppm_for_this_marker = ppm_w
                best_ppm_error_for_this_marker = ppm_error_w
            else:
                best_ppm_for_this_marker = ppm_h
                best_ppm_error_for_this_marker = ppm_error_h

            if not math.isinf(best_ppm_error_for_this_marker):
                potential_markers.append({
                    'class_name': label,
                    'pixel_per_mm': best_ppm_for_this_marker,
                    'pixel_per_mm_error': best_ppm_error_for_this_marker,
                    'bbox': (x1, y1, x2, y2),
                    'confidence': confidence
                })

        elif label == "mangosteen":
            mangosteen_boxes.append((x1, y1, x2, y2))

    print(f"    🔎 Detected classes: {class_names_detected}")

    best_marker = None
    if potential_markers:
        potential_markers.sort(key=lambda m: m['pixel_per_mm_error'])
        best_marker = potential_markers[0]

        print(f"    🏆 Best marker selected: {best_marker['class_name']} with error {best_marker['pixel_per_mm_error']:.4f}")

        bx1, by1, bx2, by2 = best_marker['bbox']
        cv2.rectangle(display_image, (bx1, by1), (bx2, by2), (0, 255, 255), scaled_line_thickness_boxes)
        cv2.putText(display_image, f"BEST MARKER: {best_marker['class_name']}",
                    (bx1, max(0, by1 - int(20 * scale_factor))),
                    cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, (0, 255, 255), scaled_font_thickness)

    if best_marker and best_marker['pixel_per_mm'] > 0:
        pixel_per_mm = best_marker['pixel_per_mm']
        pixel_per_mm_error = best_marker['pixel_per_mm_error']

        marker_mm_est = MARKER_PHYSICAL_SIZE_MM
        marker_pixel_width_used = best_marker['bbox'][2] - best_marker['bbox'][0]
        marker_pixel_height_used = best_marker['bbox'][3] - best_marker['bbox'][1]

        marker_mm_text = f"Ref: {best_marker['class_name']} {marker_pixel_width_used}x{marker_pixel_height_used}px -> {marker_mm_est:.1f}mm"
        marker_ppm_text = f"PPM: {pixel_per_mm:.2f} +/- {pixel_per_mm_error:.4f}"

        pos_x = int(10 * scale_factor)
        pos_y_line1 = int(30 * scale_factor)
        pos_y_line2 = int(pos_y_line1 + 25 * scale_factor)

        (tw1, th1), _ = cv2.getTextSize(marker_mm_text, cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, scaled_font_thickness)
        (tw2, th2), _ = cv2.getTextSize(marker_ppm_text, cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, scaled_font_thickness)
        max_tw = max(tw1, tw2)

        cv2.rectangle(display_image,
                      (pos_x - int(5 * scale_factor), pos_y_line1 - th1 - int(5 * scale_factor)),
                      (pos_x + max_tw + int(5 * scale_factor), pos_y_line2 + th2 + int(5 * scale_factor)),
                      (255, 255, 255), -1)

        cv2.putText(display_image, marker_mm_text, (pos_x, pos_y_line1), cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, (0, 0, 0), scaled_font_thickness)
        cv2.putText(display_image, marker_ppm_text, (pos_x, pos_y_line2), cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, (0, 0, 0), scaled_font_thickness)

        # Iterate through detected mangosteen boxes and estimate their size
        for i, (x1, y1, x2, y2) in enumerate(mangosteen_boxes):
            width_px = x2 - x1
            height_px = y2 - y1

            width_mm = width_px / pixel_per_mm
            height_mm = height_px / pixel_per_mm

            # ⭐ NEW: Filter out mangosteen detections that are too close to the marker's size
            if (abs(width_mm - MARKER_PHYSICAL_SIZE_MM) < SIZE_TOLERANCE_MM and
                abs(height_mm - MARKER_PHYSICAL_SIZE_MM) < SIZE_TOLERANCE_MM):
                print(f"    ➡️ Skipping mangosteen {i+1}: Detected size ({width_mm:.1f}x{height_mm:.1f}mm) is too close to marker size.")
                continue

            width_error_mm = abs(width_px * pixel_per_mm_error / (pixel_per_mm ** 2))
            height_error_mm = abs(height_px * pixel_per_mm_error / (pixel_per_mm ** 2))

            # ↔️ Horizontal width arrow (yellow)
            mid_x = (x1 + x2) // 2
            mid_y = (y1 + y2) // 2

            cv2.arrowedLine(display_image, (x1, mid_y), (x2, mid_y), (255, 255, 0), scaled_line_thickness_arrows, tipLength=scaled_tip_length)
            cv2.arrowedLine(display_image, (x2, mid_y), (x1, mid_y), (255, 255, 0), scaled_line_thickness_arrows, tipLength=scaled_tip_length)

            text_width = f"W: {width_mm:.1f} +/- {width_error_mm:.1f} mm"
            (w_text, h_text), _ = cv2.getTextSize(text_width, cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, scaled_font_thickness)
            text_x = mid_x - (w_text // 2)
            text_y = y1 - int(10 * scale_factor)

            if i == 0 and text_y < (pos_y_line2 + th2 + int(10 * scale_factor)):
                text_y = pos_y_line2 + th2 + int(20 * scale_factor)

            cv2.rectangle(display_image,
                          (text_x - int(5 * scale_factor), text_y - h_text - int(5 * scale_factor)),
                          (text_x + w_text + int(5 * scale_factor), text_y + int(5 * scale_factor)),
                          (255, 255, 255), -1)
            cv2.putText(display_image, text_width, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, (255, 255, 0), scaled_font_thickness)

            # ↕️ Vertical height arrow (green)
            cv2.arrowedLine(display_image, (mid_x, y1), (mid_x, y2), (0, 255, 0), scaled_line_thickness_arrows, tipLength=scaled_tip_length)
            cv2.arrowedLine(display_image, (mid_x, y2), (mid_x, y1), (0, 255, 0), scaled_line_thickness_arrows, tipLength=scaled_tip_length)

            text_height = f"H: {height_mm:.1f} +/- {height_error_mm:.1f} mm"
            (w2_text, h2_text), _ = cv2.getTextSize(text_height, cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, scaled_font_thickness)
            text2_x = x2 + int(10 * scale_factor)
            text2_y = mid_y

            cv2.rectangle(display_image,
                          (text2_x - int(5 * scale_factor), text2_y - h2_text - int(5 * scale_factor)),
                          (text2_x + w2_text + int(5 * scale_factor), text2_y + int(5 * scale_factor)),
                          (255, 255, 255), -1)
            cv2.putText(display_image, text_height, (text2_x, text2_y), cv2.FONT_HERSHEY_SIMPLEX, scaled_font_scale, (0, 255, 0), scaled_font_thickness)

            print(f"    ✅ Mangosteen {i+1} Estimated: W={width_mm:.1f} +/- {width_error_mm:.1f} mm, H={height_mm:.1f} +/- {height_error_mm:.1f} mm")

            # 💾 CROP CLEAN MANGOSTEEN (ORIGINAL IMAGE)
            # -------------------------------------------------------------
            clean_cropped = original_image[y1:y2, x1:x2]
            if clean_cropped.size > 0:
                base, ext = os.path.splitext(filename)
                clean_crop_filename = f"{base}_mangosteen_{i+1}_clean{ext}"
                clean_crop_filepath = os.path.join(output_clean_crops_folder, clean_crop_filename)
                cv2.imwrite(clean_crop_filepath, clean_cropped)
                print(f"    💾 Saved clean cropped image: {clean_crop_filepath}")
            else:
                print(f"    ⚠️ Failed to crop clean mangosteen {i+1}. Skipping save.")
            # -------------------------------------------------------------

            # 💾 CROP, SCALE, AND SAVE THE IMAGE FOR REGRESSION MODEL
            # -------------------------------------------------------------
            cropped_mangosteen_for_model = original_image[y1:y2, x1:x2]

            if cropped_mangosteen_for_model.size == 0:
                print(f"    ⚠️ Failed to crop mangosteen for model {i+1}. Skipping save.")
                continue

            new_width_px = int(width_mm * GLOBAL_SCALE_FACTOR)
            new_height_px = int(height_mm * GLOBAL_SCALE_FACTOR)

            if new_width_px <= 0 or new_height_px <= 0:
                print(f"    ⚠️ Dimensions of mangosteen {i+1} are too small. Skipping save.")
                continue

            resized_mangosteen = cv2.resize(cropped_mangosteen_for_model, (new_width_px, new_height_px), interpolation=cv2.INTER_AREA)

            canvas = np.full((OUTPUT_SIZE_PX, OUTPUT_SIZE_PX, 3), 255, dtype=np.uint8)

            pad_x = (OUTPUT_SIZE_PX - new_width_px) // 2
            pad_y = (OUTPUT_SIZE_PX - new_height_px) // 2

            if new_width_px > OUTPUT_SIZE_PX or new_height_px > OUTPUT_SIZE_PX:
                print(f"    ⚠️ Mangosteen {i+1} is physically too large to fit on canvas. Skipping save.")
                continue

            canvas[pad_y:pad_y+new_height_px, pad_x:pad_x+new_width_px] = resized_mangosteen

            base, ext = os.path.splitext(filename)
            output_filename = f"{base}_mangosteen_{i+1}{ext}"
            output_filepath = os.path.join(output_normalized_folder, output_filename)
            cv2.imwrite(output_filepath, canvas)

            print(f"    💾 Saved normalized image: {output_filepath}")
            # -------------------------------------------------------------

    else:
        print("    ⚠️ No suitable 'marker_square' or 'marker_circle' found with finite error – cannot estimate mm.")

    # cv2.imshow("Mangosteen Size Estimation (mm)", display_image)
    debug_path = os.path.join(output_normalized_folder, f"{base}_debug.jpg")
    cv2.imwrite(debug_path, display_image)

    key = cv2.waitKey(0)
    if key == 27:
        print("👋 ESC pressed. Exiting.")
        break

cv2.destroyAllWindows()